In [1]:
import os

In [2]:
%pwd

'c:\\Users\\anjik\\Desktop\\MLOPs_projects\\Chest_Disease_Image_Classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\anjik\\Desktop\\MLOPs_projects\\Chest_Disease_Image_Classification'

# imports

In [5]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
from cnnClassifier.entity.config_entity import DataIngestionConfig, PrepareBaseModelConfig
from cnnClassifier import logger
from cnnClassifier.config.configuration import ConfigurationManager

import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from cnnClassifier import logger

## read config.yaml, params.yaml files

In [6]:
config = read_yaml(CONFIG_FILE_PATH)
params = read_yaml(PARAMS_FILE_PATH)
print("config.yaml", config)
print("params.yaml", params)


[2024-03-16 19:41:15,726: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-16 19:41:15,729: INFO: common: yaml file: params.yaml loaded successfully]
config.yaml {'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://drive.google.com/file/d/1z0mreUtRmR-P-magILsDR3T7M6IkGXtY/view?usp=drive_link', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'prepare_base_model': {'root_dir': 'artifacts/prepare_base_model', 'base_model_path': 'artifacts/prepare_base_model/base_model.h5', 'updated_base_model_path': 'artifacts/prepare_base_model/base_model_updated.h5'}}
params.yaml {'AUGMENTATION': True, 'IMAGE_SIZE': [224, 224, 3], 'BATCH_SIZE': 16, 'INCLUDE_TOP': False, 'EPOCHS': 1, 'CLASSES': 2, 'WEIGHTS': 'imagenet', 'LEARNING_RATE': 0.01}


In [7]:
prepare_base_model_config = config.prepare_base_model
params = params
print("prepare_base_model_config:", prepare_base_model_config)
print("params:", params)

prepare_base_model_config: {'root_dir': 'artifacts/prepare_base_model', 'base_model_path': 'artifacts/prepare_base_model/base_model.h5', 'updated_base_model_path': 'artifacts/prepare_base_model/base_model_updated.h5'}
params: {'AUGMENTATION': True, 'IMAGE_SIZE': [224, 224, 3], 'BATCH_SIZE': 16, 'INCLUDE_TOP': False, 'EPOCHS': 1, 'CLASSES': 2, 'WEIGHTS': 'imagenet', 'LEARNING_RATE': 0.01}


In [8]:
prepare_base_model_config.root_dir

'artifacts/prepare_base_model'

In [9]:
prepare_base_model_config.base_model_path

'artifacts/prepare_base_model/base_model.h5'

In [10]:
prepare_base_model_config.updated_base_model_path

'artifacts/prepare_base_model/base_model_updated.h5'

# configuration

In [11]:
class ConfigurationManager:
    """
    ConfigurationManager class captures & returns configuration for components implementation
        
    """
    def __init__(self,
        # params: config.yaml, params.yaml file paths          
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        # read config, params yaml file
        self.config = read_yaml(config_filepath) # returns ConfigBox to access data easily
        self.params = read_yaml(params_filepath)
        
        # create artifact directory
        create_directories([self.config.artifacts_root])
        
          
    def get_prepare_base_model_config (self) -> PrepareBaseModelConfig:
        """
        Method: get_prepare_base_model_config
        Params:
        Returns: configurations from config.yaml, params.yaml for prepare_base_model component i.e PrepareBaseModelConfig 
        """
        logger.info("Entering get_prepare_base_model_config method of ConfigurationManager")
        prepare_base_model_config = self.config.prepare_base_model # prepare_base_model key from config.yaml
        params_config = self.params
        create_directories([prepare_base_model_config.root_dir]) # creates artifacts/prepare_base_model
        
        # returning from entity: DataIngestionConfig dataclass
        prepare_base_model_config= PrepareBaseModelConfig(
            root_dir = Path(prepare_base_model_config.root_dir),
            base_model_path= Path(prepare_base_model_config.base_model_path),
            updated_base_model_path= Path(prepare_base_model_config.updated_base_model_path),
            params_image_size= params_config.IMAGE_SIZE, 
            params_learning_rate= params_config.LEARNING_RATE, 
            params_include_top= params_config.INCLUDE_TOP,
            params_weights= params_config.WEIGHTS,
            params_classes= params_config.CLASSES   
          
        )
        logger.info("Then, exiting prepare_base_model method of ConfigurationManager")
        return prepare_base_model_config

In [12]:
configuration = ConfigurationManager(config_filepath = CONFIG_FILE_PATH,
                                                params_filepath = PARAMS_FILE_PATH)
prepare_base_model_config= configuration.get_prepare_base_model_config()
prepare_base_model_config


[2024-03-16 19:41:15,821: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-16 19:41:15,823: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-16 19:41:15,824: INFO: common: created directory at: artifacts]
[2024-03-16 19:41:15,825: INFO: 715560536: Entering get_prepare_base_model_config method of ConfigurationManager]
[2024-03-16 19:41:15,827: INFO: common: created directory at: artifacts/prepare_base_model]
[2024-03-16 19:41:15,828: INFO: 715560536: Then, exiting prepare_base_model method of ConfigurationManager]


PrepareBaseModelConfig(root_dir=WindowsPath('artifacts/prepare_base_model'), base_model_path=WindowsPath('artifacts/prepare_base_model/base_model.h5'), updated_base_model_path=WindowsPath('artifacts/prepare_base_model/base_model_updated.h5'), params_image_size=BoxList([224, 224, 3]), params_learning_rate=0.01, params_include_top=False, params_weights='imagenet', params_classes=2)

In [13]:
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

In [14]:
# write PrepareBaseModel Component
class PrepareBaseModel: 
    """
    This class gets VGG16 base model from keras, update bottom layers
    """
    
    def __init__(self, prepare_base_model_config:PrepareBaseModelConfig):
        """
        :param prepare_base_model_config: configuration for PrepareBaseModel    
        """
        self.prepare_base_model_config = prepare_base_model_config
        
    def get_base_model(self):
        """
        Method Name : get_base_model
        Description : fetch VGG16 base model from keras.applications
        Output      : 
        On Failure  :   Write an exception log and then raise an exception
        """
        logger.info("Entering get_base_model method of PrepareBaseModel")
        try:
            self.base_model= tf.keras.applications.vgg16.VGG16(
                include_top= self.prepare_base_model_config.params_include_top, 
                weights= self.prepare_base_model_config.params_weights,              
                input_shape= self.prepare_base_model_config.params_image_size         
                     
            )
            self.save_model(path= self.prepare_base_model_config.base_model_path, model=self.base_model)
            
            logger.info("Saved Vgg16 base model")
            logger.info("Then, exiting get_base_model method of PrepareBaseModel")
            
        except Exception as e:
            raise e
        
    @staticmethod
    def _prepare_full_model(base_model, classes, freeze_all, freeze_till, learning_rate):
        """
        Method Name : _prepare_full_model
        Description : update the base model (change architexture by adding custom dense layer)
        Output      : 
        On Failure  :  Write an exception log and then raise an exception
        
        """
        logger.info("Entering _prepare_full_model method of PrepareBaseModel")
        
        try: 
            logger.info("Freezing the layers")
            if freeze_all:
                for layer in base_model.layers:
                    base_model.trainable= False
            elif (freeze_all is not None) and (freeze_till > 0):
                for layer in base_model.layers[:-freeze_till]:
                    base_model.trainable= False
                
            logger.info("Flattening the layers")
            flatten_in = tf.keras.layers.Flatten()(base_model.output)  
            
            logger.info("Custom dense layer added!")
            prediction = tf.keras.layers.Dense(
                units=classes,
                activation="softmax"
            )(flatten_in) 
            
            logger.info("Preparing custom architexture of the model!")
            full_model = tf.keras.models.Model(
                inputs= base_model.input,
                outputs= prediction
            ) 
            
            logger.info("Compiling the model!")
            full_model.compile(
                optimizer= tf.keras.optimizers.SGD(learning_rate=learning_rate),
                loss= tf.keras.losses.CategoricalCrossentropy(),
                metrics=["accuracy"]
            )
        
            full_model.summary()
            logger.info("Returning full architexture of the model")
            logger.info("Then, exiting _prepare_full_model method of PrepareBaseModel")
            return full_model
        
        except Exception as e:
            raise e
        
    def get_updated_base_model(self):
        """
        Method Name : get_updated_base_model
        Description : get updated vgg16 model
        Output      : 
        On Failure  :  Write an exception log and then raise an exception
        
        """
        logger.info("Entering get_updated_base_model method of PrepareBaseModel")
        
        try: 
            self.full_model= self._prepare_full_model(
                base_model= self.base_model, 
                classes= self.prepare_base_model_config.params_classes, 
                freeze_all= True, 
                freeze_till= None, 
                learning_rate= self.prepare_base_model_config.params_learning_rate
            ) 
            self.save_model(path= self.prepare_base_model_config.updated_base_model_path, model=self.full_model)
            
            logger.info("Saved updated Vgg16 model")
            logger.info("Then, exiting get_updated_base_model method of PrepareBaseModel")
            
        except Exception as e:
            raise e
        
        
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

# Training pipeline for prepare base model component

In [15]:
try:
    logger.info("PrepareBaseModel component started")
    logger.info("Loading of PrepareBaseModel component configuration started")
    config = ConfigurationManager() # create object for ConfigurationManager class
    prepare_base_model_config = config.get_prepare_base_model_config() # obj.method() returns PrepareBaseModelConfig
    logger.info("All configuration directories, files needed for PrepareBaseModel component are ready")
    
    logger.info("PrepareBaseModel steps started")
    prepare_base_model = PrepareBaseModel(prepare_base_model_config = prepare_base_model_config) # create object for PrepareBaseModel class
    prepare_base_model.get_base_model() # obj.method()    
    prepare_base_model.get_updated_base_model()
    logger.info("OK! PrepareBaseModel component completed")
except Exception as e:
    raise e

[2024-03-16 19:41:16,115: INFO: 1053352748: PrepareBaseModel component started]
[2024-03-16 19:41:16,116: INFO: 1053352748: Loading of PrepareBaseModel component configuration started]
[2024-03-16 19:41:16,119: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-16 19:41:16,122: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-16 19:41:16,123: INFO: common: created directory at: artifacts]
[2024-03-16 19:41:16,124: INFO: 715560536: Entering get_prepare_base_model_config method of ConfigurationManager]
[2024-03-16 19:41:16,125: INFO: common: created directory at: artifacts/prepare_base_model]
[2024-03-16 19:41:16,126: INFO: 715560536: Then, exiting prepare_base_model method of ConfigurationManager]
[2024-03-16 19:41:16,127: INFO: 1053352748: All configuration directories, files needed for PrepareBaseModel component are ready]
[2024-03-16 19:41:16,128: INFO: 1053352748: PrepareBaseModel steps started]
[2024-03-16 19:41:16,129: INFO: 2663257641: Ent